In [1]:
!pip install -Uq openai-agents pydantic

In [ ]:
import os
from dotenv import load_dotenv , find_dotenv

OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export


In [ ]:
from pydantic import BaseModel
from agents import (
    Agent,
    GuardrailFunctionOutput,
    RunContextWrapper,
    Runner,
    TResponseInputItem,
    input_guardrail,
    output_guardrail,
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    RunConfig
)

In [6]:
load_dotenv(find_dotenv())

OPENROUTER_API_KEY=os.getenv('OPENROUTER_API_KEY')

In [7]:
#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url="https://openrouter.ai/api/v1",
)

model = OpenAIChatCompletionsModel(
    model="google/gemini-2.5-pro-exp-03-25:free",
    openai_client=external_client
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)

In [8]:
import nest_asyncio
nest_asyncio.apply()
import asyncio

In [9]:
math_helper = Agent(
    name="Math Helper",
    instructions="You are a math helper. Give hints or explanations for math problems, but don’t solve them completely.",
    model=model
)

In [14]:
@input_guardrail
async def check_if_math(ctx, agent, question):
    # Check if "math" is in the question (simple example)
    if "math" in question.lower():
        return GuardrailFunctionOutput(tripwire_triggered=False , output_info={"question": question})  # Good: It’s about math
    else:
        return GuardrailFunctionOutput(tripwire_triggered=True , output_info={"question": question})   # Bad: Not about math)   # Bad: Not about math

In [15]:
@output_guardrail
async def no_full_answers(ctx, agent, answer):
    # If the answer has "=", it might be solving the problem
    if "=" in answer:
        return GuardrailFunctionOutput(tripwire_triggered=True , output_info={"answer": answer})   # Bad: Looks like a full answer
    else:
        return GuardrailFunctionOutput(tripwire_triggered=False , output_info={"answer": answer})  # Good: No full answer

In [20]:
math_helper.input_guardrails = [check_if_math]
math_helper.output_guardrails = [no_full_answers]

# Step 5: Run the agent with a question
async def run_agent():
    question = "How do I solve 5x = 10?"  # Try asking something!
    try:
        result = await Runner.run(math_helper, question)
        print("Answer:", result.final_output)
    except Exception as e:
        print("Oops! Guardrail stopped it:", str(e))

In [21]:
asyncio.run(run_agent())

Oops! Guardrail stopped it: Guardrail InputGuardrail triggered tripwire
